In [12]:
import re
from re import RegexFlag

from dotenv import load_dotenv
from anthropic import Anthropic

import os


load_dotenv(os.path.abspath(os.path.join(os.getcwd(), "..")))

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))


def get_messages(prompt: str) -> str:
    response = client.messages.create(
        model="claude-3-5-sonnet-latest",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1024 * 4,
        temperature=0
    )
    return response.content[0].text

In [13]:
def generate_stepback_question(question: str):
    prompt = f"""
    당신은 물리학자입니다.

    당신의 임무는 <question> 태그 안에 있는 질문을 보고
    한 걸음 물러서서, 이 질문들의 본질적인 개념이나 원리를 추출하기 위한 추상적인 질문을 만드는 것입니다.
    이러한 stepback question은 <result> 태그 안에 작성해주세요.

    <question>
    {question}
    </question>

    <example>
    <question>
    두 개의 전하가 10cm 떨어져 있을 때, 전기력의 크기는 얼마인가요?
    </question>

    <result>
    전하 사이의 거리와 전기력의 크기 간의 상관관계를 설명하는 기본 법칙은 무엇인가요?
    </result>
    </example>
    """

    response = get_messages(prompt)

    return re.search(r"<result>(.*?)</result>", response, RegexFlag.DOTALL).group(1).strip()

In [14]:
question = generate_stepback_question("""
높이 100m 높이에서 자유낙하시켰을 때, 물체가 땅에 닿는 시간은 얼마나 걸리나요?
""")

print(question)

중력장 내에서 물체의 운동을 결정하는 기본 원리는 무엇이며, 초기 조건(높이)이 물체의 운동 시간에 어떤 영향을 미치나요?


In [19]:
from IPython.core.display import Markdown


def ask_stepback_question(stepback_question):
    return get_messages(stepback_question)


answer = ask_stepback_question(question)
display(Markdown(answer))

중력장 내 물체의 운동에 대해 상세히 설명해드리겠습니다.

기본 원리:
1. 뉴턴의 운동 법칙
- F = ma (힘 = 질량 × 가속도)
- 중력가속도 g = 9.8m/s²

2. 등가속도 운동 공식
- v = v₀ + at
- s = v₀t + (1/2)at²
- v² = v₀² + 2as

초기 높이의 영향:
1. 낙하 시간
- t = √(2h/g)
- 높이가 높을수록 낙하 시간 증가

2. 속도 변화
- 최종속도: v = √(2gh)
- 높이가 높을수록 최종속도 증가

주요 고려사항:
1. 공기저항 무시 시
- 질량과 무관한 동일 가속도
- 이상적인 포물선 운동

2. 공기저항 고려 시
- 질량, 형상에 따른 차이
- 종단속도 존재

In [28]:
def generate_question_with_stepback(question: str):
    stepback_question = generate_stepback_question(question)
    stepback_answer = ask_stepback_question(stepback_question)

    prompt = f"""
    당신은 물리학자입니다.

    아래의 <stepback_question>, <stepback_answer>태그를 먼저 읽고,
    해당 고차원적인 개념과 원리를 토대로 <original_question> 태그 안에 있는 질문을 답변해주세요.

    <stepback_question>
    {stepback_question}
    </stepback_question>

    <stepback_answer>
    {stepback_answer}
    </stepback_answer>

    <original_question>
    {question}
    </original_question>

    최종 답변은 <result> 태그 안에 작성해주세요.
    """

    messages = get_messages(prompt)
    return re.search(r"<result>(.*?)</result>", messages, RegexFlag.DOTALL).group(1).strip()

In [29]:
answer = generate_question_with_stepback("""
속력이 5km/h인 A와 속력이 2km/h B가 경주를 한다.
느린 B를 배려해주기 위해, A가 B보다 늦게 출발한다고 한다.
목적지까지의 거리가 1km인 경주일 때, A가 이기기 위해서는 최대 몇 초까지 늦게 출발해도 되는가?
""")

display(Markdown(answer))

이 문제를 단계별로 풀어보겠습니다.

1. 각 선수의 이동 시간 계산
- B의 이동 시간 = 거리 ÷ 속력
  = 1km ÷ 2km/h = 0.5시간 = 30분 = 1800초

- A의 이동 시간 = 거리 ÷ 속력
  = 1km ÷ 5km/h = 0.2시간 = 12분 = 720초

2. A가 이기기 위한 조건
- (A의 출발 시간 + A의 이동 시간) < (B의 출발 시간 + B의 이동 시간)
- (x + 720) < (0 + 1800)  [x는 A의 지연 출발 시간]
- x < 1080

따라서 A는 최대 1080초(18분)까지 늦게 출발해도 B보다 먼저 도착할 수 있습니다.

In [31]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np

NUM_CLUSTERS = 5
model = SentenceTransformer("all-MiniLM-L6-v2")


def cluster_question_semantically(questions: list[str], num_clusters: list[int]):
    embeddings = model.encode(questions)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10).fit(embeddings)

    sorted_clusters = [[] for _ in range(num_clusters)]

    for q, e, l in zip(questions, embeddings, kmeans.labels_):
        centroid = kmeans.cluster_centers_[l]
        distance = 1 - (np.dot(e, centroid) / (np.linalg.norm(e) * np.linalg.norm(centroid)))
        sorted_clusters[l].append((q, distance))

    for cluster in sorted_clusters:
        cluster.sort(key=lambda x: x[1])

    return sorted_clusters

In [32]:
questions = [
    ## 1️⃣ 역학 (Mechanics)
    "한 자동차가 시속 90 km로 똑바로 이동하고 있다. 2시간 후 자동차의 이동 거리는 몇 km인가?",
    "높이 80 m에서 공을 떨어뜨렸다. 공이 땅에 도달하는 데 걸리는 시간은 얼마인가? (중력 가속도 \( g = 9.8 \,\text{m/s}^2 \))",
    ## 2️⃣ 전자기학 (Electromagnetism)
    "두 개의 전하 \( q_1 = 2\mu C \), \( q_2 = -3\mu C \) 가 10 cm 떨어져 있다. 이들 사이의 힘의 크기는? (\( k = 9.0 \times 10^9 \,\text{N·m}^2/\text{C}^2 \))",
    "10Ω 저항에 2A의 전류가 흐를 때 전압은 얼마인가?",
    ## 3️⃣ 열역학 (Thermodynamics)
    "질량이 2kg인 물을 25°C에서 75°C까지 가열하려면 몇 J의 열이 필요한가?  (물의 비열 \( c = 4184 \,\text{J/kg·°C} \))",
    "1.0 mol의 기체가 300K에서 1.0 atm의 압력을 받을 때 부피는 얼마인가?  (\( R = 0.082 \text{ L·atm/mol·K} \))",
    ## 4️⃣ 파동 및 광학 (Waves & Optics)
    "기온이 20°C일 때 공기 중에서 소리의 속력은 약 343 m/s이다. 2초 동안 진행한 거리?",
    "공기(굴절률 \( n_1 = 1.00 \))에서 물(굴절률 \( n_2 = 1.33 \))로 빛이 30° 각도로 입사할 때 굴절각은?  (스넬 법칙: \( n_1 \sin \theta_1 = n_2 \sin \theta_2 \))",
    ## 5️⃣ 상대성이론 (Relativity)
    "우주선이 빛의 속도의 80% (\( v = 0.8c \))로 움직일 때, 지구에서 본 우주선의 10초는 우주선 내부에서는 얼마나 걸리는가?  (로렌츠 인자 \( \gamma = \frac{1}{\sqrt{1 - v^2/c^2}} \))",
    "질량이 1g 감소하면 방출되는 에너지는?",
]

clusters = cluster_question_semantically(questions, NUM_CLUSTERS)

for i, c in enumerate(clusters):
    print(f"Cluster {i+1}:")
    for q, d in c:
        print(f"{q} ({d:.2f})")
    print()

Cluster 1:
높이 80 m에서 공을 떨어뜨렸다. 공이 땅에 도달하는 데 걸리는 시간은 얼마인가? (중력 가속도 \( g = 9.8 \,	ext{m/s}^2 \)) (0.14)
두 개의 전하 \( q_1 = 2\mu C \), \( q_2 = -3\mu C \) 가 10 cm 떨어져 있다. 이들 사이의 힘의 크기는? (\( k = 9.0 	imes 10^9 \,	ext{N·m}^2/	ext{C}^2 \)) (0.14)
질량이 1g 감소하면 방출되는 에너지는? (0.15)

Cluster 2:
기온이 20°C일 때 공기 중에서 소리의 속력은 약 343 m/s이다. 2초 동안 진행한 거리? (0.12)
한 자동차가 시속 90 km로 똑바로 이동하고 있다. 2시간 후 자동차의 이동 거리는 몇 km인가? (0.18)
질량이 2kg인 물을 25°C에서 75°C까지 가열하려면 몇 J의 열이 필요한가?  (물의 비열 \( c = 4184 \,	ext{J/kg·°C} \)) (0.19)
10Ω 저항에 2A의 전류가 흐를 때 전압은 얼마인가? (0.21)

Cluster 3:
공기(굴절률 \( n_1 = 1.00 \))에서 물(굴절률 \( n_2 = 1.33 \))로 빛이 30° 각도로 입사할 때 굴절각은?  (스넬 법칙: \( n_1 \sin 	heta_1 = n_2 \sin 	heta_2 \)) (0.00)

Cluster 4:
우주선이 빛의 속도의 80% (\( v = 0.8c \))로 움직일 때, 지구에서 본 우주선의 10초는 우주선 내부에서는 얼마나 걸리는가?  (로렌츠 인자 \( \gamma = rac{1}{\sqrt{1 - v^2/c^2}} \)) (0.00)

Cluster 5:
1.0 mol의 기체가 300K에서 1.0 atm의 압력을 받을 때 부피는 얼마인가?  (\( R = 0.082 	ext{ L·atm/mol·K} \)) (0.00)



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
def select_best_question(questions: list[str], num_clusters: list[int]):
    clusters = cluster_question_semantically(questions, num_clusters)

    best_questions = []
    for i, c in enumerate(clusters):

        prompt = f"""
        당신은 물리학자입니다.

        아래의 문제를 보고, 물리학 개념을 이해하고 문제를 해결하는 방법을 차근차근 단계별로 설명해주세요.

        <question>
        {c[0]}
        </question>
        """

        response = get_messages(prompt)
        lines = response.count("\n")
        print(f"Cluster_{i+1} - Question: {c[0]} - Lines: {lines}")
        if lines > 25:
            best_questions.append((c[0], response))
            continue

    return best_questions

selected_question = select_best_question(questions, NUM_CLUSTERS)

Cluster_1 - Question: ('높이 80 m에서 공을 떨어뜨렸다. 공이 땅에 도달하는 데 걸리는 시간은 얼마인가? (중력 가속도 \\( g = 9.8 \\,\text{m/s}^2 \\))', np.float32(0.1381365)) - Lines: 29
Cluster_2 - Question: ('기온이 20°C일 때 공기 중에서 소리의 속력은 약 343 m/s이다. 2초 동안 진행한 거리?', np.float32(0.11586785)) - Lines: 24
Cluster_3 - Question: ('공기(굴절률 \\( n_1 = 1.00 \\))에서 물(굴절률 \\( n_2 = 1.33 \\))로 빛이 30° 각도로 입사할 때 굴절각은?  (스넬 법칙: \\( n_1 \\sin \theta_1 = n_2 \\sin \theta_2 \\))', np.float32(0.0)) - Lines: 26
Cluster_4 - Question: ('우주선이 빛의 속도의 80% (\\( v = 0.8c \\))로 움직일 때, 지구에서 본 우주선의 10초는 우주선 내부에서는 얼마나 걸리는가?  (로렌츠 인자 \\( \\gamma = \x0crac{1}{\\sqrt{1 - v^2/c^2}} \\))', np.float32(0.0)) - Lines: 27
Cluster_5 - Question: ('1.0 mol의 기체가 300K에서 1.0 atm의 압력을 받을 때 부피는 얼마인가?  (\\( R = 0.082 \text{ L·atm/mol·K} \\))', np.float32(0.0)) - Lines: 28


In [35]:
def format_examples(selected_question):
    formmated = []
    
    for q in enumerate(selected_question) :
        formmated.append(f"<example>\n<question>\n{q[1][0]}\n</question>\n<result>\n{q[1][1]}\n</result>\n</example>")

    return "\n\n".join(formmated)
        
def get_response_with_autoCoT(question, selected_question):
    prompt = f"""
    당신은 물리학자입니다.
    아래의 <examples> 태그 안에 작성된 예시를 참고해서
    물리학적 개념을 이해하 문제를 해결하는 방법을 파악한 후
    이를 토대로 <question> 태그 안의 질문에 대한 답변을 작성해주세요.

    <examples>
    {format_examples(selected_question)}
    </examples>

    <question>
    {question}
    </question>

    답변은 <result> 태그 안에 작성해주세요.
    """

    response = get_messages(prompt)
    return re.search(r"<result>(.*?)</result>", response, RegexFlag.DOTALL).group(1).strip()


result = get_response_with_autoCoT("반지를이 5m인 원형 트랙을 달리고 자동차의 구심 가속도는 얼마인가요? 자동차의 속력은 10m/s입니다.", selected_question)
print()


이 문제를 원운동의 구심 가속도 개념을 이용하여 해결해보겠습니다.

1) 문제 분석
- 원형 트랙의 반지름(r) = 5m
- 자동차의 속력(v) = 10m/s
- 구해야 할 것: 구심 가속도(a)

2) 사용할 공식
- 구심 가속도 공식: a = v²/r
  * 여기서 v는 속력, r은 원의 반지름
  * 구심 가속도는 원운동하는 물체가 중심을 향해 받는 가속도입니다.

3) 계산 과정
- a = v²/r
- a = (10 m/s)²/5m
- a = 100/5
- a = 20 m/s²

4) 물리적 의미
- 구심 가속도 20 m/s²는 자동차가 원운동을 유지하기 위해 중심방향으로 받는 가속도입니다.
- 이 가속도는 자동차의 속도의 방향만 바꾸고 속력은 변화시키지 않습니다.
- 이 가속도는 지구 중력가속도(약 9.8 m/s²)의 약 2배로, 상당히 큰 값입니다.

따라서 자동차의 구심 가속도는 20 m/s²입니다.
